In [45]:
import ccxt.pro as ccxtpro
import asyncio
import nest_asyncio
import pytz
from datetime import datetime
from binance_api import api_key, api_secret
import time
import ccxt
import pandas as pd
from datetime import timedelta

# Use nest_asyncio to allow async/await code to run in Jupyter Notebook
nest_asyncio.apply()

def get_current_dataframe_and_last_5_volume_sum():
    symbol = 'ETH/USDT'

    # Create an instance of the Binance exchange
    exchange = ccxt.binance({
        'options': {
            'defaultType': 'future',  # Set the default market type to 'future'
            'newUpdates': True 
        }
    })

    # Replace 'YOUR_START_TIMESTAMP' and 'YOUR_END_TIMESTAMP' with the desired timestamps
    current_time_utc = datetime.utcnow() - timedelta(hours=1)

    # Format the current time as a string in the required format ('%Y-%m-%dT%H:%M:%SZ')
    start_time_str = current_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')

    # Parse the formatted string to create the start_timestamp
    start_timestamp = exchange.parse8601(start_time_str)
    # Fetch historical OHLCV data for the specified trading pair and timeframe
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe='1m', since=start_timestamp, limit=None)

    # Create a DataFrame from the fetched data
    df = pd.DataFrame(ohlcv, columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
    df['Timestamp'] = df['Timestamp'] + timedelta(hours=8)
    df['last_5_volume_sum'] = df['Volume'].rolling(window=6, min_periods=1).sum() - df['Volume']
    last_5_volume_sum = df['last_5_volume_sum'].iloc[-1]
    # Print the DataFrame
    return df, last_5_volume_sum

async def fetch_and_print_ohlcv(exchange, symbol, df_func):
    utc_plus_8 = pytz.timezone('Asia/Shanghai')

    try:
        previous_timestamp = None  # Variable to store the previous timestamp

        while True:
            ohlcv = await exchange.watch_ohlcv(symbol, timeframe='1m')

            timestamp_utc = datetime.fromtimestamp(ohlcv[-1][0] / 1000, tz=pytz.utc)
            timestamp_utc8 = timestamp_utc.astimezone(utc_plus_8)
            current_timestamp_str = timestamp_utc8.strftime('%Y-%m-%d %H:%M:%S')
            current_volume = ohlcv[-1][5]

            # Check if the timestamp has changed
            if current_timestamp_str != previous_timestamp:
                print(f"New timestamp: {current_timestamp_str}")
                df, last_5_volume_sum = df_func()

                previous_timestamp = current_timestamp_str  # Update the previous timestamp

            if current_volume > last_5_volume_sum:
                
                print(f"last_5_volume_sum: {last_5_volume_sum}, current_volume: {current_volume}")

            print(f"time: {current_timestamp_str}, open: {ohlcv[-1][1]}, high: {ohlcv[-1][2]}, low: {ohlcv[-1][3]}, close: {ohlcv[-1][4]}, volume: {ohlcv[-1][5]}, last_5_volume_sum: {last_5_volume_sum:.2f}")

            await asyncio.sleep(1)
    except KeyboardInterrupt:
        pass
    finally:
        await exchange.close()

async def main():
    symbol = 'ETH/USDT'
    exchange = ccxtpro.binance({
        'options': {
            'defaultType': 'future',  # Set the default market type to 'future'
            'newUpdates': True 
        }
    })

    await fetch_and_print_ohlcv(exchange, symbol, get_current_dataframe_and_last_5_volume_sum)

# Run the async/await code in a Jupyter Notebook cell
asyncio.run(main())


New timestamp: 2023-09-29 01:04:00
time: 2023-09-29 01:04:00, open: 1658.4, high: 1658.7, low: 1657.7, close: 1657.7, volume: 1578.672, last_5_volume_sum: 11174.94


KeyboardInterrupt: 

In [124]:
import ccxt.pro as ccxtpro
import asyncio
import nest_asyncio
import pytz
from datetime import datetime, timedelta
import pandas as pd
from binance_api import api_key, api_secret
# Use nest_asyncio to allow async/await code to run in Jupyter Notebook
nest_asyncio.apply()

async def fetch_ohlcv_data(exchange, symbol, timeframe, since):
    try:
        ohlcv = await exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=since)
        return ohlcv
    except Exception as e:
        print(f"Error fetching OHLCV data: {e}")
        return []

def calculate_last_5_volume_sum(ohlcv_data):
    if len(ohlcv_data) >= 6:
        volumes = [entry[5] for entry in ohlcv_data[-6:]]
        last_5_volume_sum = sum(volumes) - volumes[-1]
        return last_5_volume_sum
    else:
        return 0

async def print_ohlcv_and_volume(exchange, symbol):
    utc_plus_8 = pytz.timezone('Asia/Shanghai')

    try:
        previous_timestamp = None
        last_5_volume_sum = 0

        while True:
            ohlcv_data = await fetch_ohlcv_data(exchange, symbol, timeframe='1m', since=None)
            if ohlcv_data:
                current_timestamp = ohlcv_data[-1][0] / 1000
                timestamp_utc = datetime.fromtimestamp(current_timestamp, tz=pytz.utc)
                timestamp_utc8 = timestamp_utc.astimezone(utc_plus_8)
                current_timestamp_str = timestamp_utc8.strftime('%Y-%m-%d %H:%M:%S')

                if current_timestamp_str != previous_timestamp:
                    print(f"New timestamp: {current_timestamp_str}")

                    last_5_volume_sum = calculate_last_5_volume_sum(ohlcv_data)

                    previous_timestamp = current_timestamp_str

                current_volume = ohlcv_data[-1][5]
                print(f"time: {current_timestamp_str}, open: {ohlcv_data[-1][1]}, high: {ohlcv_data[-1][2]}, low: {ohlcv_data[-1][3]}, close: {ohlcv_data[-1][4]}, volume: {ohlcv_data[-1][5]}, last_5_volume_sum: {last_5_volume_sum:.2f}")

            await asyncio.sleep(1)

    except KeyboardInterrupt:
        pass
    finally:
        await exchange.close()

async def main():
    symbol = 'ETH/USDT'
    exchange = ccxtpro.binance({
        # 'apiKey': api_key,
        # 'secret': api_secret,
        'options': {
            'defaultType': 'future',  # Set the default market type to 'future'
            'newUpdates': True
        }
    })

    await print_ohlcv_and_volume(exchange, symbol)

# Run the async/await code in a Jupyter Notebook cell
asyncio.run(main())


New timestamp: 2023-10-04 00:08:00
time: 2023-10-04 00:08:00, open: 1649.53, high: 1649.53, low: 1648.0, close: 1648.55, volume: 1194.96, last_5_volume_sum: 9656.81
time: 2023-10-04 00:08:00, open: 1649.53, high: 1649.53, low: 1648.0, close: 1648.43, volume: 1203.872, last_5_volume_sum: 9656.81
time: 2023-10-04 00:08:00, open: 1649.53, high: 1649.53, low: 1648.0, close: 1648.43, volume: 1203.872, last_5_volume_sum: 9656.81
time: 2023-10-04 00:08:00, open: 1649.53, high: 1649.53, low: 1648.0, close: 1648.43, volume: 1203.872, last_5_volume_sum: 9656.81
time: 2023-10-04 00:08:00, open: 1649.53, high: 1649.53, low: 1648.0, close: 1648.43, volume: 1209.221, last_5_volume_sum: 9656.81
time: 2023-10-04 00:08:00, open: 1649.53, high: 1649.53, low: 1648.0, close: 1648.43, volume: 1209.221, last_5_volume_sum: 9656.81
time: 2023-10-04 00:08:00, open: 1649.53, high: 1649.53, low: 1648.0, close: 1648.39, volume: 1250.732, last_5_volume_sum: 9656.81
time: 2023-10-04 00:08:00, open: 1649.53, high: 1

KeyboardInterrupt: 

In [119]:
import ccxt
from binance_api import api_key, api_secret
class Whale:
    def __init__(self, exchange, symbol, leverage=100):
        self.exchange = exchange
        self.symbol = symbol
        self.leverage = leverage
        self.test_mode = True

    def get_account_balance(self):
        if self.test_mode:
            return 100
        balance = self.exchange.fetch_balance()
        return balance['total']['USDT']

    def get_last_price(self):
        ticker = self.exchange.fetch_ticker(self.symbol)
        return ticker['last']

    def get_order_book(self):
        order_book = self.exchange.fetch_order_book(self.symbol)
        return order_book

    def get_best_bid_and_ask(self):
        order_book = self.get_order_book()
        return order_book['bids'][0][0], order_book['asks'][0][0]

    def calculate_trade_quantity(self):
        balance = self.get_account_balance()
        leveraged_balance = min(balance * self.leverage, 100000)
        quantity = leveraged_balance / self.get_last_price()
        return quantity

    def place_first_post_only_order(self, side):
        # Side could be 'buy' or 'sell'
        quantity = self.calculate_trade_quantity()
        best_bid, best_ask = self.get_best_bid_and_ask()
        price = best_bid if side == 'buy' else best_ask

        try:
            order = self.exchange.create_limit_order(
                self.symbol,
                side,
                quantity,
                price,
                {
                    'postOnly': True,
                    'timeInForce': 'PO'  
                }
            )
        except ccxt.BaseError as e:
            print(f"An error occurred: {e}")

        return order

    def place_first_market_order(self, side):
        # Side could be 'buy' or 'sell'
        quantity = self.calculate_trade_quantity()
        order = self.exchange.create_market_order(
            self.symbol,
            side,
            quantity,
            {
                'reduceOnly': False
            }
        )
        return order

    def is_position_open(self):
        position = self.exchange.fetch_positions([self.symbol])
        return len(position) > 0

    def place_second_take_profit_and_stop_loss_orders(self):
        try:
            if self.is_position_open():
                position = self.exchange.fetch_positions([self.symbol])
                if not position:
                    raise Exception("No position information found.")

                quantity = abs(float(position[0]['info']['positionAmt']))
                side = 'sell' if position[0]['side'] == 'long' else 'buy'

                entry_price = float(position[0]['info']['entryPrice'])
                take_profit_price = entry_price * (1.0007 if side == 'sell' else 0.9993)
                stop_loss_price = entry_price * (0.9993 if side == 'sell' else 1.0007)

                try:
                    take_profit_order = self.exchange.create_limit_order(
                        symbol=self.symbol,
                        side=side,
                        price=take_profit_price,
                        amount=quantity,
                        params={
                            'postOnly': True,
                            'timeInForce': 'PO',
                            'reduceOnly': True
                        }
                    )
                except ccxt.BaseError as e:
                    print(f"An error occurred in creating take-profit order: {e}")

                try:
                    stop_loss_order = self.exchange.create_order(
                        symbol=self.symbol,
                        type='stop_market',
                        side=side,
                        amount=quantity,
                        params={
                            'stopPrice': stop_loss_price,
                            'reduceOnly': True
                        }
                    )
                    
                except ccxt.BaseError as e:
                    print(f"An error occurred in creating stop-loss order: {e}")

            return take_profit_order, stop_loss_order

        except Exception as e:
            print(f"Error in placing second orders: {str(e)}")
            return None, None

    def close_open_orders(self):
        open_orders = self.exchange.fetch_open_orders(self.symbol)
        for order in open_orders:
            self.exchange.cancel_order(order['id'], self.symbol)
        return open_orders

    def close_position_with_market_order(self):
        position = self.exchange.fetch_positions([self.symbol])
        quantity = abs(float(position[0]['info']['positionAmt']))
        side = 'sell' if position[0]['side'] == 'long' else 'buy'
        order = self.exchange.create_market_order(
            self.symbol,
            side,
            quantity,
            {
                'reduceOnly': True
            }
        )
        return order

    def close_position_and_orders(self):
        try:
            self.close_open_orders()
        except Exception as e:
            print(f"Error in closing open orders: {str(e)}")

        try:
            self.close_position_with_market_order()
        except Exception as e:
            print(f"Error in closing position: {str(e)}")


# Replace with your API key and secret
symbol = 'ETH/USDT'

exchange = ccxt.binance({
    'options': {
        'defaultType': 'future',
        'newUpdates': True 
    },
    'apiKey': api_key,
    'secret': api_secret
})
exchange.set_sandbox_mode(True)

# Create an instance of the Whale class
test = Whale(exchange, symbol)
test.calculate_trade_quantity()


6.066451914268901

In [121]:
test.place_first_market_order('buy')
test.place_second_take_profit_and_stop_loss_orders()



({'info': {'orderId': '1180957573',
   'symbol': 'ETHUSDT',
   'status': 'NEW',
   'clientOrderId': 'x-xcKtGhcud379af1d6468c61ea77c62',
   'price': '1649.79',
   'avgPrice': '0.00',
   'origQty': '6.066',
   'executedQty': '0.000',
   'cumQty': '0.000',
   'cumQuote': '0.00000',
   'timeInForce': 'GTX',
   'type': 'LIMIT',
   'reduceOnly': True,
   'closePosition': False,
   'side': 'SELL',
   'positionSide': 'BOTH',
   'stopPrice': '0.00',
   'workingType': 'CONTRACT_PRICE',
   'priceProtect': False,
   'origType': 'LIMIT',
   'priceMatch': 'NONE',
   'selfTradePreventionMode': 'NONE',
   'goodTillDate': '0',
   'updateTime': '1696348646012'},
  'id': '1180957573',
  'clientOrderId': 'x-xcKtGhcud379af1d6468c61ea77c62',
  'timestamp': 1696348646012,
  'datetime': '2023-10-03T15:57:26.012Z',
  'lastTradeTimestamp': None,
  'lastUpdateTimestamp': 1696348646012,
  'symbol': 'ETH/USDT:USDT',
  'type': 'limit',
  'timeInForce': 'PO',
  'postOnly': True,
  'reduceOnly': True,
  'side': 'sell

In [120]:
test.close_position_and_orders()


Error in closing position: binance amount of ETH/USDT:USDT must be greater than minimum amount precision of 3
